In [1]:
import os
import cv2
# import tkinter
import matplotlib
import numpy as np
# matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
# %matplotlib tk

First, the pointLoad function loads in the specified image and prompts to hand pick a set amount of points. TODO: store points for future runs to disable popup's and embed images for ease of documentation

In [2]:
def pointLoad(imgpath,pts = 0, imgpass = True):

    if os.path.exists(imgpath):
        img = cv2.imread(imgpath)
        img=cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        if pts:
            plt.imshow(img)
            ptMatrix = plt.ginput(pts)
            # plt.close()
            if imgpass:
                return img, np.array(ptMatrix).astype(int)
            else:
                return np.array(ptMatrix).astype(int)    
        else:
            return img
    else:
        print("Invalid image path")
        return
    
            

getH creates a homography matrix between the source and destination photos, instead of using svd, I opted to construct the matrix with an explicit final h33 param = 1 and use least squares, something svd does internally anyway but I want to be difficult...

In [3]:
def getH(dest, src):
    p = np.array([0,0,0,0,0,0,0,0,1])
    # if len(dest) < 5 and len(src) < 5:
        

    for i in range(len(dest)):
        pi = np.array([[-src[i][0],-src[i][1],-1,0,0,0,src[i][0]*dest[i][0],src[i][1]*dest[i][0],dest[i][0]],
        [0,0,0,-src[i][0],-src[i][1],-1,src[i][0]*dest[i][1],src[i][1]*dest[i][1],dest[i][1]]])
        p = np.vstack((pi,p))

  
    x = np.zeros((len(dest)*2,1))
    x = np.vstack((x,np.array([1])))
    H, residuals, rank, s = np.linalg.lstsq(p,x)
    H = H.reshape((3,3))
    

    
    return H
        


Pano is a vestigiel function from when I thought pt1 of this homework let you use a detector for finding points on the panorama =( I'm leaving it here to use for pt2 so not all is lost

In [4]:
def pano(image_paths):
    imgs = []
    imgs_gray = []
    
    orbvals={"keypts":[],"descrpt":[],"matches":[]}
    orb = cv2.ORB_create()
    # sift = cv2.xfeatures2d.SIFT_create(nfeatures=200)
    for i in range(len(image_paths)):
        imgs.append(cv2.imread(image_paths[i]))
        imgs[i]=cv2.resize(imgs[i],(0,0),fx=0.4,fy=0.4)
        imgs_gray.append(cv2.cvtColor(imgs[i],cv2.COLOR_BGR2GRAY))
        keypt, desc = orb.detectAndCompute(imgs_gray[i],None)
        # keypt, desc = sift.detectAndCompute(imgs_gray[i], None)
        orbvals["keypts"].append(keypt)
        orbvals["descrpt"].append(desc)
        # print((orbvals["keypts"]))
        # cv2.imshow("image",cv2.drawKeypoints(imgs[i], orbvals["keypts"][i], None, (255, 0, 255)))


    bf = cv2.BFMatcher_create(cv2.NORM_HAMMING)    
    for i in range(len(imgs)-1):
        orbvals["matches"].append(bf.knnMatch(orbvals["descrpt"][i], orbvals["descrpt"][i+1],k=2))
    print(orbvals['matches'])
    cv2.imshow(draw_matches(imgs[0],orbvals["keypts"][0],imgs[1],orbvals["keypts"][1],orbvals["matches"][0]))


draw_matches function to visualize points

In [5]:
def draw_matches(img1, keypoints1, img2, keypoints2, matches):
  r, c = img1.shape[:2]
  r1, c1 = img2.shape[:2]

  # Create a blank image with the size of the first image + second image
  output_img = np.zeros((max([r, r1]), c+c1, 9), dtype='uint8')
  output_img[:r, :c, :] = np.dstack([img1, img1, img1])
  output_img[:r1, c:c+c1, :] = np.dstack([img2, img2, img2])

  # Go over all of the matching points and extract them
  for match in matches:
    (x1, y1) = keypoints1[img1_idx].pt
    (x2, y2) = keypoints2[img2_idx].pt

    # Draw circles on the keypoints
    cv2.circle(output_img, (int(x1),int(y1)), 4, (0, 255, 255), 1)
    cv2.circle(output_img, (int(x2)+c,int(y2)), 4, (0, 255, 255), 1)

    # Connect the same keypoints
    cv2.line(output_img, (int(x1),int(y1)), (int(x2)+c,int(y2)), (0, 255, 255), 1)
    
  return output_img

function to paste the source image onto the destination, the implementation itself is a little hack-y

In [ ]:
def pasteImages(img1,img1pts,img2,img2pts):
    H = getH(img1pts,img2pts)
    im_out = cv2.warpPerspective(img2,H,(img1.shape[1],img1.shape[0]))
    mask = np.zeros(img1.shape[:2], dtype="uint8")
    cv2.fillConvexPoly(mask, np.int32([img1pts]),color=255)
    cv2.fillConvexPoly(img1, np.int32([img1pts]),color=0)
    masked =cv2.bitwise_and(im_out,im_out,mask=mask)
    img3 = cv2.add(img1,masked)
    # img3=cv2.cvtColor(img3, cv2.COLOR_BGR2RGB)
    return img3, H


extract 4 points from the images

In [ ]:
img1, img1pts = pointLoad("image1.jpg",4)

img2, img2pts = pointLoad("image2.jpg",4)



paste the images together

In [ ]:

img3, H3 = pasteImages(img1,img1pts,img2,img2pts)

and display...

In [ ]:

plt.imshow(img3)

plt.show()

This process can also be repeated with as many points as youd like due to the getH function performing least squares on the images, here we will do just that with 8 points from each image...

In [ ]:
img1New, img1ptsNew = pointLoad("image1.jpg",8)
img2New, img2ptsNew = pointLoad("image2.jpg",8)


In [ ]:
img3New, H3New = pasteImages(img1New,img1ptsNew,img2New,img2ptsNew)

In [ ]:
plt.imshow(img3New)

plt.show()

I can compare the two H matrix's and see how they differ...

In [ ]:
print(H3)
print("~~~~")
print(H3New)

Clearly, both Homographies differ while still yielding similar transoformations. In part, this is due to my bad clicking accuracy between images. A more major factor however, is that the transofrms are scaled differently.
If you observe the value at h33, the scale factors differ wildly, given that that h33 = wk where w is an arbitrary scale factor and k is 1. This difference for us does not matter fortunately as these transformations are scale invariant.

Now lets perform this 8 point homography again but for a new image!

In [ ]:
img4, img4pts = pointLoad("image3.jpg",8)

In [ ]:
img5, H5 = pasteImages(img1New,img1ptsNew,img4,img4pts)

plt.imshow(img5)


Now lets try some panorama stitching. This will work in much the same way as before but with some extra complications for our final output image.

In [6]:

image_paths=['1.jpg','2.jpg','3.jpg','4.jpg','5.jpg']
# left={'pts':[],'h':[]}
# right={'pts':[],'h':[]}
# # get points/homographies for left-bound images
# for n in [2,1]:
#     destpts = pointLoad(image_paths[n],4,False)
#     left['pts'].append(destpts)
#     srcpts = pointLoad(image_paths[n-1],4,False)
#     left['pts'].append(srcpts)
#     left['h'].append(getH(destpts,srcpts))

# for n in [2,3]:
#     destpts = pointLoad(image_paths[n],4,False)
#     right['pts'].append(destpts)
#     srcpts = pointLoad(image_paths[n+1],4,False)
#     right['pts'].append(srcpts)
#     right['h'].append(getH(destpts,srcpts))
    


left= {'pts':[np.array([[ 66, 333],
       [119, 330],
       [124, 428],
       [ 68, 434]]), np.array([[568, 361],
       [611, 353],
       [616, 446],
       [569, 452]]), np.array([[344, 456],
       [487, 439],
       [488, 488],
       [347, 499]]), np.array([[750, 436],
       [899, 410],
       [900, 463],
       [751, 480]])], 'h': [np.array([[ 1.09302538e+01,  2.20491068e-01, -5.76948603e+03],
       [ 4.60920510e+00,  9.74051812e+00, -3.51831044e+03],
       [ 1.05745646e-02,  2.35356950e-03,  1.00000000e+00]]), np.array([[ 2.26918546e+00,  2.88756032e-01, -1.16863724e+03],
       [ 5.01158722e-01,  2.16272647e+00, -4.45061492e+02],
       [ 8.81876514e-04,  5.84231271e-04,  1.00000000e+00]])]}
right={'pts': [np.array([[ 993,  297],
       [1057,  292],
       [1059,  385],
       [ 994,  388]]), np.array([[481, 376],
       [535, 375],
       [539, 461],
       [483, 461]]), np.array([[633, 378],
       [171, 458]])], 'h': [np.array([[ 9.87533238e-01,  2.33782564e-01,  4.33166039e+02],
       [-1.27974513e-01,  1.17262990e+00, -8.14343553e+01],
       [-1.89906519e-04,  2.51166116e-04,  1.00000000e+00]]), np.array([[ 3.23123829e-01, -2.92838171e-01,  5.31994939e+02],
       [-2.61739097e-01,  5.04467128e-01,  1.44761482e+02],
       [-4.77680542e-04, -4.37126071e-04,  1.00000000e+00]])]}


Now we have an array of the left homographies and of the right as well as points.

In [7]:

print(left)
print('~~~')
print(right)

{'pts': [array([[ 66, 333],
       [119, 330],
       [124, 428],
       [ 68, 434]]), array([[568, 361],
       [611, 353],
       [616, 446],
       [569, 452]]), array([[344, 456],
       [487, 439],
       [488, 488],
       [347, 499]]), array([[750, 436],
       [899, 410],
       [900, 463],
       [751, 480]])], 'h': [array([[ 1.09302538e+01,  2.20491068e-01, -5.76948603e+03],
       [ 4.60920510e+00,  9.74051812e+00, -3.51831044e+03],
       [ 1.05745646e-02,  2.35356950e-03,  1.00000000e+00]]), array([[ 2.26918546e+00,  2.88756032e-01, -1.16863724e+03],
       [ 5.01158722e-01,  2.16272647e+00, -4.45061492e+02],
       [ 8.81876514e-04,  5.84231271e-04,  1.00000000e+00]])]}
~~~
{'pts': [array([[ 993,  297],
       [1057,  292],
       [1059,  385],
       [ 994,  388]]), array([[481, 376],
       [535, 375],
       [539, 461],
       [483, 461]]), array([[633, 378],
       [171, 458]])], 'h': [array([[ 9.87533238e-01,  2.33782564e-01,  4.33166039e+02],
       [-1.27974513e-01

Before we do any transforms, we need to relate our outer edges of the panorama to the central image (image 3). this could either be done by warping the outermost image to the one inner image and then warping those to the center.
This method, however, is flawed as it will result in some drift from the error in the homographies.

A better way to apraoch the edge images is to dot the outer homoography with the inner to create a direct relation from the outer image to the central one.

In [9]:
images=[]
for image in image_paths:
    img = (pointLoad(image))
    cv2.imshow("image",img)
    images.append(img)

# np.zeros()
# im_out = cv2.warpPerspective(images[1],left['h'][0],(5000,5000))
# cv2.imshow('warped',im_out)
# # cv2.addweighted(orig1, alpha, orig2, (1- alpha), 0, output);